In [1]:
import stock_object 
import stock_price_extraction as spe
import balance_sheet_extraction as ble 
import income_statement_extraction as ise
import share_holders_extraction as she
import cashflow_extraction as ce
import warnings
import pandas as pd
from utility_methods import Utility as u
warnings.filterwarnings('ignore')

C:\Users\shiva.reddy\AppData\Roaming\Python\Python310\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
nse_company_file = 'C:\\Users\\shiva.reddy\\Personal_Projects\\NSE_Analysis\\NSE_Analysis\\NSE_Project\Input\\NSE_Listed_Companies_SectorWise_Sample.csv'
df_nse = pd.read_csv(nse_company_file,sep='~')
df_nse =  df_nse.head(3)
df_nse.head()

,Ticker,Description,Sector,Industry,Exchange,nse_ticker
0,LTIM,LTIMINDTREE LTD,Commercial Services,Miscellaneous Commercial Services,NSE,LTIM.NS
1,KPITTECH,KPIT TECHNOLOGIES,Technology Services,Packaged Software,NSE,KPITTECH.NS
2,HDFC,HOUSING DEVEL FIN,Finance,Finance/Rental/Leasing,NSE,HDFC.NS


In [3]:
# stores the output dataframes of each companies for later consolidation
balance_sheet_yearly_dfs = []
balance_sheet_quarterly_dfs = []

income_statement_yearly_dfs = []
income_statement_quarterly_dfs = []

cashflow_yearly_dfs = []
cashflow_quartely_dfs = []

price_history_dfs = []

for index,row in df_nse.iterrows():
    ticker = row['Ticker']
    print('Company : ',ticker)
    stock = stock_object.Stock(row['nse_ticker'])
    stock_obj = stock.getStockObject()
    
    # price history 
    stock_price_obj = spe.StockPriceExtraction(stock_obj , ticker)
    df_stock_price_company_wise = stock_price_obj.getPriceHistory()
    
    
    # balance statement yearly
    balance_sheet_obj =  ble.BalanceSheetExtraction(stock_obj)
    df_balance_sheet_yearly =  balance_sheet_obj.getBalanceSheetYearly()
    df_balance_sheet_yearly =  u.getTransposedMatrix(df_balance_sheet_yearly)
    df_balance_sheet_yearly= u.setTickerField(df_balance_sheet_yearly,ticker)
    
    # balance statement quarterly
    df_balance_sheet_quartely = balance_sheet_obj.getBalanceSheetQuarterly()
    df_balance_sheet_quartely = u.getTransposedMatrix(df_balance_sheet_quartely)
    df_balance_sheet_quartely=  u.setTickerField(df_balance_sheet_quartely,ticker)
    
    # income statement yearly
    income_stmt_obj =  ise.IncomeStatementExtraction(stock_obj)
    df_income_stmt_yearly = income_stmt_obj.getIncomeStatementYearly()
    df_income_stmt_yearly = u.getTransposedMatrix(df_income_stmt_yearly)
    df_income_stmt_yearly= u.setTickerField(df_income_stmt_yearly,ticker)
    
    # income stmt quarterly
    df_income_stmt_quarterly = income_stmt_obj.getIncomeStatementQuarterly()
    df_income_stmt_quarterly =  u.getTransposedMatrix(df_income_stmt_quarterly)
    df_income_stmt_quarterly=u.setTickerField(df_income_stmt_quarterly,ticker)
    
    # cashflow statements yearly
    cashflow_obj = ce.CashFlowExtraction(stock_obj)
    df_cashflow_yearly = cashflow_obj.getCashFlowYearly()
    df_cashflow_yearly = u.getTransposedMatrix(df_cashflow_yearly)
    df_cashflow_yearly=u.setTickerField(df_cashflow_yearly,ticker)
    
    # cashflow stmt quarterly
    df_cashflow_quarterly = income_stmt_obj.getIncomeStatementQuarterly()
    df_cashflow_quarterly = u.getTransposedMatrix(df_cashflow_quarterly)
    df_cashflow_quarterly= u.setTickerField(df_cashflow_quarterly,ticker)
    
    
    # list of dataframe objects for later consolidation to single df
    price_history_dfs.append(df_stock_price_company_wise)
    balance_sheet_yearly_dfs.append(df_balance_sheet_yearly)
    balance_sheet_quarterly_dfs.append(df_balance_sheet_quartely)
    
    income_statement_yearly_dfs.append(df_income_stmt_yearly)
    income_statement_quarterly_dfs.append(df_income_stmt_quarterly)

    cashflow_yearly_dfs.append(df_cashflow_yearly)
    cashflow_quartely_dfs.append(df_cashflow_quarterly)
    


    
    

Company :  LTIM
Company :  KPITTECH
Company :  HDFC


In [4]:
df_stock_prices_consolidated = u.consolidate(price_history_dfs)
df_balance_sheets_yearly_consolidated = u.consolidate(balance_sheet_yearly_dfs)
df_balance_sheets_quarterly_consolidated = u.consolidate(balance_sheet_quarterly_dfs) 



In [6]:
df_stock_prices_consolidated.head()

,Date,ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2021-01-01 00:00:00+05:30,LTIM,3530.135376,3623.196479,3503.125491,3574.878174,285207,0.0,0.0
1,2021-01-04 00:00:00+05:30,LTIM,3604.158996,3778.394610,3584.686578,3716.015869,373949,0.0,0.0
2,2021-01-05 00:00:00+05:30,LTIM,3715.677351,4057.674191,3675.089978,3990.463623,840857,0.0,0.0
3,2021-01-06 00:00:00+05:30,LTIM,4007.520001,4019.116393,3802.650399,3823.813721,597076,0.0,0.0
4,2021-01-07 00:00:00+05:30,LTIM,3863.579678,3893.488659,3739.836463,3752.302490,363617,0.0,0.0


In [5]:
df_balance_sheets_yearly_consolidated.head()

,index,ticker,Treasury Shares Number,Ordinary Shares Number,Share Issued,Total Debt,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,...,Finished Goods,Work In Process,Raw Materials,Net Debt,Derivative Product Liabilities,Investments And Advances,Trading Securities,Investmentsin Associatesat Cost,Investment Properties,Receivables
0,2023-03-31,LTIM,16058.0,295806721.0,295822779.0,15412000000.0,150469000000.0,167174000000.0,117032000000.0,150469000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-03-31,LTIM,NaN,175270156.0,175270156.0,13911000000.0,128011000000.0,143391000000.0,98463000000.0,128011000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-03-31,LTIM,NaN,174750608.0,174750608.0,7983000000.0,63793000000.0,73448000000.0,54990000000.0,63793000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2023-03-31,KPITTECH,3797418.0,270346390.0,274143808.0,2865260000.0,4140420000.0,17007480000.0,3335180000.0,4140420000.0,...,155420000.0,99000000.0,333080000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-03-31,KPITTECH,4127233.0,270016575.0,274143808.0,2272260000.0,11127240000.0,13121470000.0,8065000000.0,11127240000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_balance_sheets_yearly_consolidated.to_json(orient='records')

'[{"index":1680220800000,"ticker":"LTIM","Treasury Shares Number":16058.0,"Ordinary Shares Number":295806721.0,"Share Issued":295822779.0,"Total Debt":15412000000.0,"Tangible Book Value":150469000000.0,"Invested Capital":167174000000.0,"Working Capital":117032000000.0,"Net Tangible Assets":150469000000.0,"Capital Lease Obligations":14159000000.0,"Common Stock Equity":165921000000.0,"Total Capitalization":165921000000.0,"Total Equity Gross Minority Interest":165992000000.0,"Minority Interest":71000000.0,"Stockholders Equity":165921000000.0,"Other Equity Interest":4839000000.0,"Retained Earnings":154501000000.0,"Additional Paid In Capital":4033000000.0,"Capital Stock":296000000.0,"Common Stock":296000000.0,"Total Liabilities Net Minority Interest":68969000000.0,"Total Non Current Liabilities Net Minority Interest":14143000000.0,"Other Non Current Liabilities":null,"Tradeand Other Payables Non Current":1773000000.0,"Non Current Deferred Taxes Liabilities":147000000.0,"Long Term Debt And C

In [5]:
df_stock_prices_consolidated.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker
0,2021-01-01 00:00:00+05:30,3530.135135,3623.196232,3503.125251,3574.877930,285207,0.0,0.0,LTIM
1,2021-01-04 00:00:00+05:30,3604.158760,3778.394362,3584.686343,3716.015625,373949,0.0,0.0,LTIM
2,2021-01-05 00:00:00+05:30,3715.677351,4057.674191,3675.089978,3990.463623,840857,0.0,0.0,LTIM
3,2021-01-06 00:00:00+05:30,4007.520001,4019.116393,3802.650399,3823.813721,597076,0.0,0.0,LTIM
4,2021-01-07 00:00:00+05:30,3863.580181,3893.489165,3739.836950,3752.302979,363617,0.0,0.0,LTIM


In [7]:
df_balance_sheets_yearly_consolidated.ticker.unique()

array(['LTIM', 'KPITTECH', 'HDFC'], dtype=object)

In [12]:
df_balance_sheets_yearly_consolidated.head()

,2023-03-31 00:00:00,2022-03-31 00:00:00,2021-03-31 00:00:00,ticker
Treasury Shares Number,16058.0,NaN,NaN,LTIM
Ordinary Shares Number,295806721.0,175270156.0,174750608.0,LTIM
Share Issued,295822779.0,175270156.0,174750608.0,LTIM
Total Debt,15412000000.0,13911000000.0,7983000000.0,LTIM
Tangible Book Value,150469000000.0,128011000000.0,63793000000.0,LTIM


In [17]:
df_balance_sheets_yearly_consolidated.columns.drop('ticker')

Index([2023-03-31 00:00:00, 2022-03-31 00:00:00, 2021-03-31 00:00:00], dtype='object')

In [27]:
df_balance_sheets_yearly_consolidated.T

,Treasury Shares Number,Ordinary Shares Number,Share Issued,Total Debt,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,Capital Lease Obligations,Common Stock Equity,...,Construction In Progress,Other Properties,Machinery Furniture Equipment,Buildings And Improvements,Land And Improvements,Prepaid Assets,Receivables,Taxes Receivable,Cash And Cash Equivalents,Cash Financial
2023-03-31 00:00:00,16058.0,295806721.0,295822779.0,15412000000.0,150469000000.0,167174000000.0,117032000000.0,150469000000.0,14159000000.0,165921000000.0,...,119000000.0,20189800000.0,6767400000.0,18879700000.0,5102900000.0,19815000000.0,3366200000.0,41631100000.0,34629100000.0,3670600000.0
2022-03-31 00:00:00,NaN,175270156.0,175270156.0,13911000000.0,128011000000.0,143391000000.0,98463000000.0,128011000000.0,13392000000.0,142872000000.0,...,13500000.0,667400000.0,5999400000.0,17487900000.0,4550300000.0,17159200000.0,42950400000.0,32616700000.0,26993000000.0,2020800000.0
2021-03-31 00:00:00,NaN,174750608.0,174750608.0,7983000000.0,63793000000.0,73448000000.0,54990000000.0,63793000000.0,7569000000.0,73034000000.0,...,97600000.0,511700000.0,4537900000.0,14264300000.0,4550300000.0,7427900000.0,39117900000.0,29202800000.0,30977500000.0,3816000000.0
ticker,LTIM,LTIM,LTIM,LTIM,LTIM,LTIM,LTIM,LTIM,LTIM,LTIM,...,HDFC,HDFC,HDFC,HDFC,HDFC,HDFC,HDFC,HDFC,HDFC,HDFC


In [ ]:
df_balance_sheets_yearly_consolidated.head()

In [14]:
df_balance_sheets_yearly_consolidated.T.reset_index()

,index,Treasury Shares Number,Ordinary Shares Number,Share Issued,Total Debt,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,Capital Lease Obligations,...,Construction In Progress,Other Properties,Machinery Furniture Equipment,Buildings And Improvements,Land And Improvements,Prepaid Assets,Receivables,Taxes Receivable,Cash And Cash Equivalents,Cash Financial
0,2023-03-31 00:00:00,16058.0,295806721.0,295822779.0,15412000000.0,150469000000.0,167174000000.0,117032000000.0,150469000000.0,14159000000.0,...,119000000.0,20189800000.0,6767400000.0,18879700000.0,5102900000.0,19815000000.0,3366200000.0,41631100000.0,34629100000.0,3670600000.0
1,2022-03-31 00:00:00,NaN,175270156.0,175270156.0,13911000000.0,128011000000.0,143391000000.0,98463000000.0,128011000000.0,13392000000.0,...,13500000.0,667400000.0,5999400000.0,17487900000.0,4550300000.0,17159200000.0,42950400000.0,32616700000.0,26993000000.0,2020800000.0
2,2021-03-31 00:00:00,NaN,174750608.0,174750608.0,7983000000.0,63793000000.0,73448000000.0,54990000000.0,63793000000.0,7569000000.0,...,97600000.0,511700000.0,4537900000.0,14264300000.0,4550300000.0,7427900000.0,39117900000.0,29202800000.0,30977500000.0,3816000000.0
3,ticker,LTIM,LTIM,LTIM,LTIM,LTIM,LTIM,LTIM,LTIM,LTIM,...,HDFC,HDFC,HDFC,HDFC,HDFC,HDFC,HDFC,HDFC,HDFC,HDFC


In [ ]:
df_balance_sheets_yearly_consolidated = df.drop(df.index[df.index == 'ticker'])